In [2]:
from sklearn.datasets import fetch_rcv1
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

In [3]:
X, y = fetch_rcv1(shuffle=True, return_X_y=True)

In [8]:
type(rcv1)

NameError: name 'rcv1' is not defined

In [3]:
class SOM:
    def __init__(self, map_size, lr=0.1):
        """
        :param map_size: [map_w, map_h, f]
        """
        self.map = np.random.random(size=map_size)
        self.lr0 = lr
        self.lr = lr
        
        self.R0 = map_size[0]//2
        self.R = self.R0

        self.scores = np.zeros(shape=(self.map.shape[0], self.map.shape[1], 3))
    
    def find_winner(self, x):
        repeated_x = np.tile(x, [self.map.shape[0], self.map.shape[1], 1])
        dists = np.sum((self.map - repeated_x)**2, axis=2)
        
        winner = np.unravel_index(np.argmin(dists), dists.shape)
        
        return winner
    
    def get_NS(self, winner):
        NS = np.zeros(shape=(self.map.shape[0], self.map.shape[1]))

        iw, jw = winner[0], winner[1]
        
        NS[iw, jw] = 1
        
        for r in range(1, int(self.R)):
            if iw - r >= 0:
                NS[iw - r, jw] = 1/r   
            if iw + r < self.map.shape[0]:
                NS[iw + r, jw] = 1/r
                
            if jw - r >= 0:
                NS[iw, jw - r] = 1/r
            if jw + r < self.map.shape[1]:
                NS[iw, jw + r] = 1/r
                
        return NS
        
    
    def update_weights(self, x, n_strength):
        NS = np.tile(n_strength, [self.map.shape[2], 1, 1]).transpose() # ERROR
        
        repeated_x = np.tile(x, [self.map.shape[0], self.map.shape[1], 1])
        Delta = repeated_x - self.map
        
        self.map += self.lr * np.multiply(NS, Delta)
    
    def train(self, X, epochs=1000, error_threshold=10**-20):
        Js = []
        for epoch in range(epochs):
            prev_map = np.copy(self.map)
            for row in range(X.shape[0]):
                x = X[row, :]
                x = x.toarray()
                winner = self.find_winner(x) # winner = [5, 23]
                neighbors = self.get_NS(winner)
                self.update_weights(x, neighbors)
            
            self.lr = self.lr0 * (1 - epoch/epochs)
            self.R = self.R0 * (1 - epoch/epochs)
            
            Js.append(np.linalg.norm(prev_map - self.map))
            
#             if epoch % 100 == 0:
            print("Iteration: %d, LR: %f, R: %f, J: %f" %(epoch, self.lr, self.R, Js[-1]))
            
            if Js[-1] < error_threshold:
                print("MIN CHANGE")
                break
            
        return Js

    def visualize(self, X, y):
        for i in range(len(X)):
            x = X[i, :]
            winner = self.find_winner(x)
            iw, jw = winner[0], winner[1]

            if y[i] == 1:


In [4]:
X_small = X[:100]

In [13]:
som_net = SOM(map_size=[9, 9, X_small.shape[1]])
Js = som_net.train(X_small, epochs=10)

Iteration: 0, LR: 0.100000, R: 4.000000, J: 0.000000
MIN CHANGE


In [ ]:
plt.plot(Js)
plt.show()

In [6]:
X.shape

(804414, 47236)

In [7]:
y.shape


(804414, 103)